In [3]:
import pandas as pd
import json

In [14]:
with open("index_rune", "r") as fp:
    idx = json.load(fp)
with open("impact_categories", "r") as fp:
    impact_category = json.load(fp)


[['EF v3.1 EN15804', 'acidification', 'accumulated exceedance (AE)'],
 ['EF v3.1 EN15804', 'climate change', 'global warming potential (GWP100)'],
 ['EF v3.1 EN15804',
  'ecotoxicity: freshwater',
  'comparative toxic unit for ecosystems (CTUe)'],
 ['EF v3.1 EN15804',
  'energy resources: non-renewable',
  'abiotic depletion potential (ADP): fossil fuels'],
 ['EF v3.1 EN15804',
  'eutrophication: freshwater',
  'fraction of nutrients reaching freshwater end compartment (P)'],
 ['EF v3.1 EN15804',
  'eutrophication: marine',
  'fraction of nutrients reaching marine end compartment (N)'],
 ['EF v3.1 EN15804',
  'eutrophication: terrestrial',
  'accumulated exceedance (AE)'],
 ['EF v3.1 EN15804',
  'human toxicity: carcinogenic',
  'comparative toxic unit for human (CTUh)'],
 ['EF v3.1 EN15804',
  'human toxicity: non-carcinogenic',
  'comparative toxic unit for human (CTUh)'],
 ['EF v3.1 EN15804',
  'ionising radiation: human health',
  'human exposure efficiency relative to u235'],
 ['E

In [34]:
ic_lst = []
for ic in impact_category:
    str_temp = '('
    for icc in ic:
        if icc != ic[-1]:
            str_temp += f'{icc}, '
        else:
            str_temp += f'{icc})'
        # print(icc)
    ic_lst.append(str_temp)
    # print(str_temp)

ic_lst

['(EF v3.1 EN15804, acidification, accumulated exceedance (AE))',
 '(EF v3.1 EN15804, climate change, global warming potential (GWP100))',
 '(EF v3.1 EN15804, ecotoxicity: freshwater, comparative toxic unit for ecosystems (CTUe))',
 '(EF v3.1 EN15804, energy resources: non-renewable, abiotic depletion potential (ADP): fossil fuels)',
 '(EF v3.1 EN15804, eutrophication: freshwater, fraction of nutrients reaching freshwater end compartment (P))',
 '(EF v3.1 EN15804, eutrophication: marine, fraction of nutrients reaching marine end compartment (N))',
 '(EF v3.1 EN15804, eutrophication: terrestrial, accumulated exceedance (AE))',
 '(EF v3.1 EN15804, human toxicity: carcinogenic, comparative toxic unit for human (CTUh))',
 '(EF v3.1 EN15804, human toxicity: non-carcinogenic, comparative toxic unit for human (CTUh))',
 '(EF v3.1 EN15804, ionising radiation: human health, human exposure efficiency relative to u235)',
 '(EF v3.1 EN15804, land use, soil quality index)',
 '(EF v3.1 EN15804, mate

In [64]:
file_name_sub = r'Results\Ananas sub - CONSQ.xlsx'
df = pd.read_excel(file_name_sub)
# Convert JSON strings back to lists for all columns
df = df.applymap(lambda x: json.loads(x) if isinstance(x, str) and x.startswith('[') else x)
df = df.set_axis(idx)
df.columns = ic_lst
df

C:\Users\ruw\AppData\Local\Temp\1\ipykernel_28472\3576165090.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: json.loads(x) if isinstance(x, str) and x.startswith('[') else x)


,"(EF v3.1 EN15804, acidification, accumulated exceedance (AE))","(EF v3.1 EN15804, climate change, global warming potential (GWP100))","(EF v3.1 EN15804, ecotoxicity: freshwater, comparative toxic unit for ecosystems (CTUe))","(EF v3.1 EN15804, energy resources: non-renewable, abiotic depletion potential (ADP): fossil fuels)","(EF v3.1 EN15804, eutrophication: freshwater, fraction of nutrients reaching freshwater end compartment (P))","(EF v3.1 EN15804, eutrophication: marine, fraction of nutrients reaching marine end compartment (N))","(EF v3.1 EN15804, eutrophication: terrestrial, accumulated exceedance (AE))","(EF v3.1 EN15804, human toxicity: carcinogenic, comparative toxic unit for human (CTUh))","(EF v3.1 EN15804, human toxicity: non-carcinogenic, comparative toxic unit for human (CTUh))","(EF v3.1 EN15804, ionising radiation: human health, human exposure efficiency relative to u235)","(EF v3.1 EN15804, land use, soil quality index)","(EF v3.1 EN15804, material resources: metals/minerals, abiotic depletion potential (ADP): elements (ultimate reserves))","(EF v3.1 EN15804, ozone depletion, ozone depletion potential (ODP))","(EF v3.1 EN15804, particulate matter formation, impact on human health)","(EF v3.1 EN15804, photochemical oxidant formation: human health, tropospheric ozone concentration increase)","(EF v3.1 EN15804, water use, user deprivation potential (deprivation-weighted water consumption))"
Steel cylinder - sc 1,"[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),..."
nitrous oxide production - sc 1,"[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N..."
Use phase basecase - sc 1,"[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 227....","[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 1.33...","[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 0.0]]"
EoL 50L cylinder - FU

In [73]:
for index, row in df.iterrows():
    # print()
    col_count = 0
    for row_element in row:
        print(index, impact_category[col_count][1])
        col_count += 1
        for proc in row_element:
            if 'transport' in proc[0]:    
                print(proc[0], proc[1])
                
        print()

Steel cylinder - sc 1 acidification
'transport, freight, lorry 16-32 metric ton, EURO6' (ton kilometer, RER, None) 3.434578940649671e-07
'transport, freight, inland waterways, barge tanker' (ton kilometer, RER, None) 3.699990455344564e-08

Steel cylinder - sc 1 climate change
'transport, freight, lorry 16-32 metric ton, EURO6' (ton kilometer, RER, None) 0.00017103437440246975
'transport, freight, inland waterways, barge tanker' (ton kilometer, RER, None) 4.314397150294659e-06

Steel cylinder - sc 1 ecotoxicity: freshwater
'transport, freight, lorry 16-32 metric ton, EURO6' (ton kilometer, RER, None) 0.0011495134804026183
'transport, freight, inland waterways, barge tanker' (ton kilometer, RER, None) 2.1933242220265765e-05

Steel cylinder - sc 1 energy resources: non-renewable
'transport, freight, lorry 16-32 metric ton, EURO6' (ton kilometer, RER, None) 0.002423623626017584
'transport, freight, inland waterways, barge tanker' (ton kilometer, RER, None) 5.1770118247744625e-05

Steel cyl